In [10]:
import pandas as pd
import numpy as np
from time import time

POIs = pd.read_csv('AllPOI Locatoion Only.csv').values
_, dimension = POIs.shape
POIs = POIs.tolist()

In [54]:
n, d = 10, 2

In [12]:
# Task 1
def MBR(POINTS):
    points = np.array(POINTS)
    length, dnum = points.shape
    num_iter, MBR = length // 2, []
    mod = (length - num_iter * 2) == 1
    for j in range(dnum):
        xls = points[:, j]
        x_min, x_max = float('inf'), float('-inf')
        for i in range(num_iter):
            l = 2 * i
            r, xl = l + 1, xls[l]
            xr = xls[r]
            if xl > xr:
                if xl > x_max:
                    x_max = xl
                if xr < x_min:
                    x_min = xr
            else:
                if xr > x_max:
                    x_max = xr
                if xl < x_min:
                    x_min = xl
        if mod:
            x = xls[length - 1]
            if x > x_max:
                x_max = x
            if x < x_min:
                x_min = x
        MBR.append([x_min, x_max])
    return MBR

def split(x, index = None):
    # Split a node to 2 ones.
    # A heuristic method to reduce the covered area of subtrees. 
    num_points, points = len(x), np.array(x)
    if index == None:
        index = list(range(num_points))
    d22 = np.zeros((num_points, num_points))
    for i in range(num_points - 1):
        t = points[i] - points[i+1:]
        d22[i][i+1:] = np.sum(t*t, axis = 1)
    a = np.argmax(d22)
    a, b = divmod(a, num_points)
    # Find the pair of points with the furthest distance. 
    points = x
    part1, part2 = [points[a]], [points[b]]
    index1, index2 = [index[a]], [index[b]]
    for i in range(num_points):
        # Assign each points to the nearest cluster. 
        if i < a:
            if d22[i][a] >= d22[i][b]:
                part2.append(points[i])
                index2.append(index[i])
            else:
                part1.append(points[i])
                index1.append(index[i])
        elif i > a and i < b:
            if d22[a][i] >= d22[i][b]:
                part2.append(points[i])
                index2.append(index[i])
            else:
                part1.append(points[i])
                index1.append(index[i])
        elif i > b:
            if d22[a][i] >= d22[b][i]:
                part2.append(points[i])
                index2.append(index[i])
            else:
                part1.append(points[i])
                index1.append(index[i])
    return index1, part1, MBR(part1), index2, part2, MBR(part2)

def insert(rtree, point, i, r):
    # Insert a point to the R Tree by recursion.
    # i: where to insert?
    # r: the index of root.
    node = rtree[i]
    height = node[0]
    if height == 0:
        # Height = 0 means a leaf. 
        bucket = node[2]
        bucket.append(point)
        if len(bucket) > n:
            # Split a leaf if there is more than n points. 
            _, node[2],node[1], _, b2, r2 = split(bucket)
            rtree.append([0, r2, b2])
            if i == r:
                rtree.append([1, [[min(node[1][j][0], r2[j][0]), max(node[1][j][1], r2[j][1])] for j in range(dimension)], [i, len(rtree) - 1]])
                # The form of a node: [height, MBR, subtrees/bucket]
                return len(rtree) - 1, len(rtree) - 1
            return len(rtree) - 1, r
            # Return the index of new leaf. 
        else:
            # Add the point to the leaf and update the MBR.  
            bound = node[1]
            node[1] = [[min(bound[j][0], point[j]), max(bound[j][1], point[j])] for j in range(dimension)]
            return -1, r
            # -1 means no new leaf/subtree are created. 
    else:
        bound, s = node[1], float('inf')
        node[1] = [[min(bound[j][0], point[j]), max(bound[j][1], point[j])] for j in range(dimension)]
        # Update the MBR. 
        for subtree in node[2]:
            s_new = 1.0
            for pj, bj in zip(point, rtree[subtree][1]):
                s_new *= (max(pj, bj[1]) - min(pj, bj[0]))
            if s_new < s:
                s = s_new
                select = subtree
        ref, _ = insert(rtree, point, select, r)
        # Select the subtree with the smallest MBR after insertion, then insert the points to it.
        if ref < 0:
            # ref < 0 means no new subtree are created. 
            return -1, r
        else:
            subtrees = node[2]
            subtrees.append(ref)
            t1, t2 = node[1], rtree[ref][1]
            node[1] = [[min(t1[j][0], t2[j][0]), max(t1[j][1], t2[j][1])] for j in range(dimension)]
            # Add the index of new subtree to the node and update the MBR
            if len(subtrees) > d:
                # If there are more than d subtrees, then split it. 
                center = []
                for subtree in subtrees:
                    bound = rtree[subtree][1]
                    center.append([(r[0] + r[1]) / 2 for r in bound])
                st1, _, r1, st2, _, r2 = split(center, subtrees)
                # Divide the subtrees into 2 groups. 
                for subtree in st1:
                    bound = rtree[subtree][1]
                    r1 = [[min(bound[j][0], r1[j][0]), max(bound[j][1], r1[j][1])] for j in range(dimension)]
                node[1], node[2] = r1, st1
                for subtree in st2:
                    bound = rtree[subtree][1]
                    r2 = [[min(bound[j][0], r2[j][0]), max(bound[j][1], r2[j][1])] for j in range(dimension)]
                rtree.append([height, r2, st2])
                # Add the new node to the tree. 
                if i == r:
                    # If the root is splitted, then the index of new root will be also returned. 
                    rtree.append([height + 1, [[min(r1[j][0], r2[j][0]), max(r1[j][1], r2[j][1])] for j in range(dimension)], [i, len(rtree) - 1]])
                    return len(rtree) - 1, len(rtree) - 1
                else:
                    return len(rtree) - 1, r
            else:
                return -1, r

def intersect(bound1, bound2):
    # Check whether two MBRs intersect. 
    intersection = [[max(bound1[j][0], bound2[j][0]), min(bound1[j][1], bound2[j][1])] for j in range(dimension)]
    for intersection_axis in intersection:
        if intersection_axis[1] < intersection_axis[0]:
            return False, None
    # If intersect, the intersection will be also returned. 
    return True, intersection

In [55]:
# Task 2
rtree, r, start = [[0, [[float('inf'), float('-inf')] for _ in range(dimension)], []]], 0, time()
# initialization: RTree has only one leaf, which is also the root. 
# r: The index of root.
for point in POIs:
    _, r = insert(rtree, point, r, r)
print('Time of Construction:', time() - start, 's')
# Insert each point in POIs to R Tree. 
mbr = rtree[r][1]
num_non_leaf, num_leaf, distri, overlap, non_leafs = 0, 0, [0, 0, 0, 0], set(), []
for i in range(len(rtree)):
    node = rtree[i]
    if node[0] == 0:
        num_leaf += 1
        num_points = len(node[2])
        utili = num_points / n
        if utili < 0.25:
            distri[0] += 1
        elif utili < 0.5:
            distri[1] += 1
        elif utili < 0.75:
            distri[2] += 1
        else:
            distri[3] += 1
    else:
        num_non_leaf += 1
        non_leafs.append(node)

for i in range(len(non_leafs)):
    node1, uninserted = non_leafs[i], True
    bound1 = node1[1]
    for j in range(i + 1, len(non_leafs)):
        node2 = non_leafs[j]
        is_intersect, _ = intersect(bound1, node2[1])
        if is_intersect:
            overlap.add(j)
            if uninserted:
                overlap.add(i)
                uninserted = False

del non_leafs
print('Number of Non-leaf:', num_non_leaf)
print('Number of Overlapped Non-leaf:', len(overlap))
del overlap
print('Number of Leaf:', num_leaf)
print('Height:', rtree[r][0])
print('Utilisation:')
for i in range(4):
    print(25*i,'%-',25*(i+1),'%:',100 * distri[i] / num_leaf, '%')

Time of Construction: 15.104040384292603 s
Number of Non-leaf: 40554
Number of Overlapped Non-leaf: 40554
Number of Leaf: 26849
Height: 19
Utilisation:
0 %- 25 %: 5.8996610674513015 %
25 %- 50 %: 14.08618570524042 %
50 %- 75 %: 35.12607545904876 %
75 %- 100 %: 44.88807776825953 %


In [52]:
# Task 3
def nearest_point(rtree, i, point, nearest, distance, minmaxdist_min, calculated, visited, pruned):
    # Find the nearest point to a point by pruning. 
    node = rtree[i]
    if node[0] == 0:
        # For a leaf, linear scan. 
        points = node[2]
        for pt in points:
            d22 = sum([(pt[j] - point[j])**2 for j in range(dimension)])
            if d22 < distance:
                distance = d22
                nearest = pt
        return nearest, distance, minmaxdist_min, calculated + len(points), visited, pruned
    else:
        for subtree in node[2]:
            visited += 1
            bound, mindist_point, mindist = rtree[subtree][1], [], 0
            for a, boundj in zip(point, bound):
                # Calculate the MinDist. 
                alower, aupper = boundj[0], boundj[1]
                if a > aupper:
                    mindist_point.append(aupper)
                    mindist += (a - aupper) ** 2
                elif a < alower:
                    mindist_point.append(alower)
                    mindist += (alower - a) ** 2
                else:
                    mindist_point.append(a)
            if mindist < minmaxdist_min:
                # Prune if MinDist >= the MinMaxDist to a MBR.
                for boundj, a, ma in zip(bound, point, mindist_point):
                    # Calculate the MinMaxDist.
                    alower, aupper = boundj[0], boundj[1]
                    if ma == aupper:
                        minmaxdist = mindist + (a - alower) ** 2 - (a - aupper) ** 2
                    elif ma == alower:
                        minmaxdist = mindist - (a - alower) ** 2 + (a - aupper) ** 2
                    else:
                        minmaxdist = mindist + max(aupper - a, a - alower) ** 2
                    if minmaxdist < minmaxdist_min:
                        minmaxdist_min = minmaxdist
                        # Update the the lower bound of MinMaxDist
                if mindist < distance:
                    # Prune if MinDist >= the distance between a pair of points.
                    nearest, distance, minmaxdist_min, calculated, visited, pruned = nearest_point(rtree, subtree, point, nearest, distance, minmaxdist_min, calculated, visited, pruned)
                else:
                    pruned += 1
            else:
                pruned += 1
        return nearest, distance, minmaxdist_min, calculated, visited, pruned

In [37]:
# Task4
random_points = [[bound[0] + (bound[1] - bound[0]) * np.random.rand() for bound in mbr] for _ in range(10)]

In [56]:
for point in random_points:
    print('Location:', point)
    nearest, distance, _, calculated, visited, pruned = nearest_point(rtree, r, point, None, float('inf'), float('inf'), 0, 1, 0)
    print('Nearest Point:', nearest)
    print('Visited:', visited)
    print('Calculated:', calculated)
    print('Distance:', np.sqrt(distance))
    print('Pruned:', pruned)
    print('\n')

Location: [115.94531732236037, 40.14053076146112]
Nearest Point: [115.95083100000001, 40.13997]
Visited: 12280
Calculated: 4930
Distance: 0.0055421200393132985
Pruned: 4429


Location: [117.08356531663472, 40.58166560819375]
Nearest Point: [117.08436999999999, 40.588656]
Visited: 3982
Calculated: 1693
Distance: 0.007036554051750568
Pruned: 1330


Location: [117.45729216318621, 40.966206148763426]
Nearest Point: [117.440496, 40.660835]
Visited: 1014
Calculated: 127
Distance: 0.3058327150500307
Pruned: 387


Location: [116.90969223049798, 39.61344865740029]
Nearest Point: [116.88479299999999, 39.679266999999996]
Visited: 7714
Calculated: 1480
Distance: 0.0703706323828654
Pruned: 3168


Location: [116.81976040043281, 40.76009235124847]
Nearest Point: [116.800606, 40.766368]
Visited: 1676
Calculated: 451
Distance: 0.020156260148973072
Pruned: 606


Location: [116.69408458192285, 40.99315209269646]
Nearest Point: [116.628786, 41.007536]
Visited: 438
Calculated: 129
Distance: 0.0668640530513